<a href="https://colab.research.google.com/github/hospino/Cursos-Platzi/blob/main/cargarROADMAPBorde_Hybrid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# Celda Cero: Inicialización de GEE
import ee
PROYECTO_ID = "gen-lang-client-0528470346"

try:
    # Intenta inicializar directamente con el proyecto.
    ee.Initialize(project=PROYECTO_ID)
    print("✅ Earth Engine inicializado.")

except Exception as e:
    # Si falla, es porque el token expiró. Ejecuta la autenticación de nuevo.
    print("❌ Error de inicialización. Ejecutando ee.Authenticate()...")
    ee.Authenticate()
    # Si la autenticación es exitosa, intenta inicializar de nuevo.
    ee.Initialize(project=PROYECTO_ID)
    print("✅ Autenticación y reinicialización exitosa.")

✅ Earth Engine inicializado.


In [7]:

# Celda 2: Obtención y Procesamiento de la Imagen Sentinel-2 (Recorte por Estado)

import ee
# Asegúrate de que GEE esté inicializado (Celda Cero) antes de ejecutar esta.

# 1. Definir la ubicación y el Asset ID
# Usando el ID del Asset que subiste a tu proyecto
ASSET_ID_MERIDA = 'projects/gen-lang-client-0528470346/assets/MeridaBorde'
lon = -71.146
lat = 8.587
punto_merida = ee.Geometry.Point(lon, lat)

try:
    # Carga la geometría usando el ID del Asset.
    # Nota: El Asset ID de proyecto requiere el prefijo 'projects/'
    area_de_estudio = ee.FeatureCollection(ASSET_ID_MERIDA).geometry()
    print(f"✅ Geometría del Estado Mérida ({ASSET_ID_MERIDA}) cargada. Usando recorte político.")
except Exception as e:
    print(f"❌ ERROR: El Asset ID no es válido. Usando el círculo de 30km como respaldo. Error: {e}")
    area_de_estudio = punto_merida.buffer(30000)

# 2. Definir el rango de tiempo y Colección Sentinel-2
fecha_inicio = '2024-01-01'
fecha_fin = '2024-03-01'

# Filtros
coleccion_s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate(fecha_inicio, fecha_fin) \
    .filterBounds(area_de_estudio) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

# 3. Crear el mosaico compuesto y RECORTARLO con el límite del Estado
imagen_compuesta = coleccion_s2.median().clip(area_de_estudio)

print("✅ Datos Sentinel-2 procesados (listos para el recorte poligonal).")

✅ Geometría del Estado Mérida (projects/gen-lang-client-0528470346/assets/MeridaBorde) cargada. Usando recorte político.
✅ Datos Sentinel-2 procesados (listos para el recorte poligonal).


In [9]:
 # Celda 3: Visualización con Base Satelital (HYBRID) y Centrado Forzado

from IPython.display import display, Markdown
import geemap

# Parámetros de la ubicación de Mérida
lon = -71.146
lat = 8.587
zoom_level = 9 # Usamos un zoom ligeramente más alejado para ver todo el límite del estado

# 1. Parámetros de Visualización (Color Natural)
vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'] # Bandas Sentinel-2: Red, Green, Blue
}

# 2. Crear el objeto Mapa (Usando el basemap estable 'HYBRID')
m = geemap.Map(basemap='HYBRID')

# 3. FORZAR el centrado del mapa en Mérida (Anula el centrado predeterminado de África)
m.setCenter(lon, lat, zoom_level)

# 4. Añadir la capa de imagen satelital de GEE (Recortada al polígono)
m.addLayer(
    ee_object=imagen_compuesta,
    vis_params=vis_params,
    name='Sentinel-2 Compuesta (Recorte Poligonal)'
)

# 5. Mostrar el mapa
m.add_layer_control()
display(Markdown("### ✅ Resultado: Recorte Poligonal del Estado Mérida"))
m

### ✅ Resultado: Recorte Poligonal del Estado Mérida

Map(center=[8.587, -71.146], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…